Bioinformatikos lab.1 darbas 
Vilius Macijauskas - 1911074

Sekos turi būti sukeltos į Google drive:
/content/drive/MyDrive/bioinformatika1

In [2]:
import itertools
!pip install biopython
from Bio.Seq import Seq
from Bio import SeqIO
from google.colab import drive
drive.mount('/content/drive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
START_CODONS = ['ATG']
END_CODONS = ['TAG', 'TAA', 'TGA']
LETTERS = ['A','T','C','G']

def generateDicodons(list):
  dicodons = []
  for firstC in list:
    for secondC in list:
      dicodons.append(firstC + secondC)
  return dicodons

ALL_CODONS = [''.join(comb) for comb in itertools.product(LETTERS, repeat=3)]
ALL_DICODONS = generateDicodons(ALL_CODONS)

In [4]:
# Šis metodas nebuvo panaudotas sekų ieškojime, nes paprastesnis variantas buvo tai padaryti einant per sekas pirmą kartą (pagal atsakymą iš MS Teams)
# Palieku, kad užpildytų pirmą punktą
def findCodonPositions(sequence):
  start_codons = {'frame0': [], 'frame1': [], 'frame2': []}
  end_codons = {'frame0': [], 'frame1': [], 'frame2': []}

  seq_pos = 0
  current_frame = 0
  while seq_pos < len(sequence) - 2:

    if sequence[seq_pos] + sequence[seq_pos + 1] + sequence[seq_pos + 2] in START_CODONS:
      start_codons.get('frame' + str(current_frame)).append(seq_pos)

    if sequence[seq_pos] + sequence[seq_pos + 1] + sequence[seq_pos + 2] in END_CODONS:
      end_codons.get('frame' + str(current_frame)).append(seq_pos)

    current_frame = (current_frame + 1) % 3
    seq_pos += 1

  return start_codons, end_codons

In [5]:
def findPossibleCombinations(sequence):
  combinations = {'frame0': [], 'frame1': [], 'frame2': []}

  seq_pos = 0
  current_frame = 0
  start_pos = [None,None,None]
  while seq_pos < len(sequence) - 2:

    if sequence[seq_pos] + sequence[seq_pos + 1] + sequence[seq_pos + 2] in START_CODONS:
      if start_pos[current_frame] == None:
        start_pos[current_frame] = seq_pos

    if sequence[seq_pos] + sequence[seq_pos + 1] + sequence[seq_pos + 2] in END_CODONS:
      if start_pos[current_frame] != None:
        combinations.get('frame' + str(current_frame)).append({'start': start_pos[current_frame], 'end': seq_pos})
        start_pos[current_frame] = None

    current_frame = (current_frame + 1) % 3
    seq_pos += 1

  return combinations

In [6]:
def filterLongPairs(combinations, min_length):
  filtered_combinations = {'frame0': [], 'frame1': [], 'frame2': []}

  for frame in range(0, 3):
    for combo in combinations.get('frame' + str(frame)):
      if combo.get('end') - combo.get('start') + 3 > min_length:
        filtered_combinations.get('frame' + str(frame)).append(combo)

  return filtered_combinations

In [7]:
def extractSequences(sequence, combinations):
  sequences = []

  for frame in range(0, 3):
    for combo in combinations.get('frame' + str(frame)):
      sequences.append(sequence[combo.get('start') + 3:combo.get('end')])

  return sequences

In [8]:
def findCodonsFrequency(sequences):
  frequencies = {}
  codonCount = 0
  i = 0

  for codon in ALL_CODONS:
      frequencies[codon] = 0

  for clipping in sequences:
    while i < len(clipping) - 2:
      codon = clipping[i:i + 3]
      frequencies[codon] += 1
      codonCount += 1
      i += 1

  for codon in ALL_CODONS:
      frequencies[codon] = round(frequencies[codon] / codonCount, 3)

  return frequencies

def findDicodonsFrequency(sequences):
  frequencies = {}
  dicodonCount = 0
  i = 0

  for dicodon in ALL_DICODONS:
    frequencies[dicodon] = 0

  for clipping in sequences:
    while i < len(clipping) - 5:
      dicodon = clipping[i:i + 6]
      frequencies[dicodon] += 1
      dicodonCount += 1
      i += 1

  for dicodon in ALL_DICODONS:
      frequencies[dicodon] = round(frequencies[dicodon] / dicodonCount, 3)

  return frequencies

In [9]:
def compareFrequencies(frequencies1, frequencies2, codons):
  distance = 0
  for codon in codons:
    distance += abs(frequencies1[codon] - frequencies2[codon])
  return distance

In [15]:
def computeFrequencies(sequence):
  reverse_sequence = str(Seq(sequence).reverse_complement())
  sequence = str(sequence)

  # surandame sekų pradžias ir pabaigas skirtinguose frame'uose, kurių pradžios yra toliausiai nuo pabaigų
  combinations = findPossibleCombinations(sequence);
  rev_combinations = findPossibleCombinations(reverse_sequence);

  # atfiltruojame per trumpas sekas
  combinations = filterLongPairs(combinations, 100)
  rev_combinations = filterLongPairs(rev_combinations, 100)

  # iš sekos sudaromos iškarpos
  sequences = extractSequences(sequence, combinations)
  rev_sequences = extractSequences(sequence, rev_combinations)

  # randami kodonų dažniai 
  codon_frequencies = findCodonsFrequency(sequences + rev_sequences)

  # randami dikodonų dažniai 
  dicodon_frequencies = findDicodonsFrequency(sequences + rev_sequences)

  return {'codon_frequencies': codon_frequencies, 'dicodon_frequencies': dicodon_frequencies}


In [16]:
# Buvo naudojamas surikiuoti kodonų/dikodonų sąrašus pagal dydžius ataskaitai
def findMostCommonStrings(frequencies, all_strings):
  sorted_list = []
  for x in all_strings:
    sorted_list.append((x, frequencies.get(x)))
  return sorted(sorted_list, key = lambda x: x[1], reverse = True)

In [17]:
data = [{'filename':'bacterial1.fasta','virus':'Lactococcus_phage'},
        {'filename':'bacterial2.fasta','virus':'KM389305.1'},
        {'filename':'bacterial3.fasta','virus':'NC_028697.1'},
        {'filename':'bacterial4.fasta','virus':'KC821626.1'},
        {'filename':'mamalian1.fasta','virus':'coronavirus'},
        {'filename':'mamalian2.fasta','virus':'adenovirus'},
        {'filename':'mamalian3.fasta','virus':'U18337.1'},
        {'filename':'mamalian4.fasta','virus':'herpesvirus'}]
        
frequencies = {}

for data_element in data:
  sequence = SeqIO.read("/content/drive/MyDrive/bioinformatika1/" + data_element['filename'], "fasta")
  frequencies[data_element['virus']] = computeFrequencies(sequence.seq)

In [20]:
for x in data:
  print(x['virus'],end = " ")
  for y in data:
    print(round(compareFrequencies(frequencies[x['virus']]['codon_frequencies'],frequencies[y['virus']]['codon_frequencies'], ALL_CODONS),3),end = " ")
  print()

Lactococcus_phage 0.0 0.426 0.285 0.366 0.332 0.426 0.43 0.67 
KM389305.1 0.426 0.0 0.419 0.606 0.466 0.28 0.598 0.39 
NC_028697.1 0.285 0.419 0.0 0.463 0.329 0.353 0.525 0.607 
KC821626.1 0.366 0.606 0.463 0.0 0.4 0.61 0.424 0.824 
coronavirus 0.332 0.466 0.329 0.4 0.0 0.408 0.38 0.664 
adenovirus 0.426 0.28 0.353 0.61 0.408 0.0 0.59 0.35 
U18337.1 0.43 0.598 0.525 0.424 0.38 0.59 0.0 0.8 
herpesvirus 0.67 0.39 0.607 0.824 0.664 0.35 0.8 0.0 


In [21]:
for x in data:
  print(x['virus'],end = " ")
  for y in data:
    print(round(compareFrequencies(frequencies[x['virus']]['dicodon_frequencies'],frequencies[y['virus']]['dicodon_frequencies'], ALL_DICODONS),3),end = " ")
  print()

Lactococcus_phage 0.0 1.689 1.17 1.044 0.982 1.229 1.336 1.859 
KM389305.1 1.689 0.0 1.801 1.827 1.723 1.742 1.829 2.064 
NC_028697.1 1.17 1.801 0.0 1.23 1.06 1.333 1.47 1.923 
KC821626.1 1.044 1.827 1.23 0.0 0.872 1.297 1.258 1.947 
coronavirus 0.982 1.723 1.06 0.872 0.0 1.119 1.18 1.793 
adenovirus 1.229 1.742 1.333 1.297 1.119 0.0 1.493 1.744 
U18337.1 1.336 1.829 1.47 1.258 1.18 1.493 0.0 2.091 
herpesvirus 1.859 2.064 1.923 1.947 1.793 1.744 2.091 0.0 
